In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import Font
from IPython.display import FileLink


In [ ]:
# Define the Gaussian function with baseline
def gaussian_with_baseline(x, amp1, cen1, wid1, amp2, cen2, wid2, baseline):
    return (amp1 * np.exp(-(x - cen1)**2 / (2 * wid1**2)) +
            amp2 * np.exp(-(x - cen2)**2 / (2 * wid2**2)) +
            baseline)


In [ ]:
# Generate synthetic data
np.random.seed(0)
x_data = np.linspace(0, 100, 200)
y_data = (30 * np.exp(-(x_data - 30)**2 / (2 * 10**2)) +
          20 * np.exp(-(x_data - 70)**2 / (2 * 8**2)) +
          5 + np.random.normal(0, 2, x_data.size))


In [ ]:
# Fit the data
initial_guess = [25, 25, 10, 15, 65, 10, 5]
params, _ = curve_fit(gaussian_with_baseline, x_data, y_data, p0=initial_guess)
y_fit = gaussian_with_baseline(x_data, *params)
residuals = y_data - y_fit


In [ ]:
# Individual Gaussians and baseline
gauss1 = params[0] * np.exp(-(x_data - params[1])**2 / (2 * params[2]**2))
gauss2 = params[3] * np.exp(-(x_data - params[4])**2 / (2 * params[5]**2))
baseline = np.full_like(x_data, params[6])


In [ ]:
# Plotting
fig, axs = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
axs[0].plot(x_data, y_data, 'b.', label='Observed')
axs[0].plot(x_data, y_fit, 'r-', label='Fitted')
axs[0].legend()
axs[0].set_title('Observed vs Fitted')

axs[1].plot(x_data, gauss1, 'g--', label='Gaussian 1')
axs[1].plot(x_data, gauss2, 'm--', label='Gaussian 2')
axs[1].plot(x_data, baseline, 'k--', label='Baseline')
axs[1].legend()
axs[1].set_title('Individual Gaussians and Baseline')

axs[2].plot(x_data, residuals, 'k.')
axs[2].axhline(0, color='gray', linestyle='--')
axs[2].set_title('Residuals')
axs[2].set_xlabel('X')

plt.tight_layout()
plt.savefig("gaussian_fit_plot.png")


In [ ]:
# Prepare Excel data
df_results = pd.DataFrame({
    'X': x_data,
    'Y Observed': y_data,
    'Y Fitted': y_fit,
    'Residuals': residuals
})

mse = np.mean(residuals**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(residuals))

df_stats = pd.DataFrame({
    'Statistic': ['MSE', 'RMSE', 'MAE'],
    'Value': [mse, rmse, mae]
})

# Create Excel workbook
wb = Workbook()
ws1 = wb.active
ws1.title = "Fit Results"

for r in dataframe_to_rows(df_results, index=False, header=True):
    ws1.append(r)

tab = Table(displayName="FitResults", ref=f"A1:D{len(df_results)+1}")
style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True)
tab.tableStyleInfo = style
ws1.add_table(tab)

ws2 = wb.create_sheet(title="Statistics")
for r in dataframe_to_rows(df_stats, index=False, header=True):
    ws2.append(r)

for col in ws1.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = max_length + 2

for col in ws2.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws2.column_dimensions[col[0].column_letter].width = max_length + 2

wb.save("gaussian_fit_results.xlsx")


In [ ]:
# Provide download links
print("Download Excel file:")
display(FileLink("gaussian_fit_results.xlsx"))

print("Download plot image:")
display(FileLink("gaussian_fit_plot.png"))
